#### Produce constructor standings

In [0]:
%run "../includes/configuration"


In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.functions import *

In [0]:
dbutils.widgets.text("p_file_date","2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

Find race years for which the data is to be reprocessed

In [0]:
race_results_df = spark.read.format("delta").load(F"{presentation_folder_path}/race_results")\
    .filter(f"file_date = '{v_file_date}'")
  

In [0]:
race_year_list = df_column_to_list(race_results_df, "race_year")

In [0]:
display(race_results_df)

race_id,race_year,race_name,race_date,circuit_location,driver_name,driver_id,driver_number,driver_nationality,team,grid,fastest_lap,race_time,points,position,file_date,current_date
1053,2021,Emilia Romagna Grand Prix,2021-04-18,Imola,Mick Schumacher,854,null,German,Haas F1 Team,18,58,\N,0.0,16,2021-04-18,2025-04-10T23:40:54.560+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18,Imola,Nikita Mazepin,853,null,Russian,Haas F1 Team,19,55,\N,0.0,17,2021-04-18,2025-04-10T23:40:54.560+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18,Imola,Yuki Tsunoda,852,null,Japanese,AlphaTauri,20,49,+73.184,0.0,12,2021-04-18,2025-04-10T23:40:54.560+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18,Imola,Nicholas Latifi,849,6,Canadian,Williams,14,null,\N,0.0,null,2021-04-18,2025-04-10T23:40:54.560+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18,Imola,George Russell,847,63,British,Williams,12,28,\N,0.0,null,2021-04-18,2025-04-10T23:40:54.560+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18,Imola,Lando Norris,846,4,British,McLaren,7,63,+23.702,15.0,3,2021-04-18,2025-04-10T23:40:54.560+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18,Imola,Charles Leclerc,844,16,Monegasque,Ferrari,4,60,+25.579,12.0,4,2021-04-18,2025-04-10T23:40:54.560+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18,Imola,Pierre Gasly,842,10,French,AlphaTauri,5,52,+52.818,6.0,7,2021-04-18,2025-04-10T23:40:54.560+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18,Imola,Antonio Giovinazzi,841,99,Italian,Alfa Romeo,17,57,\N,0.0,14,2021-04-18,2025-04-10T23:40:54.560+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18,Imola,Lance Stroll,840,18,Canadian,Aston Martin,10,59,+56.909,4.0,8,2021-04-18,2025-04-10T23:40:54.560+0000


In [0]:
from pyspark.sql.functions import sum, when, col, count, desc , rank
from pyspark.sql.window import Window

In [0]:
from pyspark.sql.functions import col
race_results_df = spark.read.format("delta")\
    .load(F"{presentation_folder_path}/race_results")\
    .filter( col("race_year").isin(race_year_list))

In [0]:
team_standings_df = race_results_df\
    .groupBy( "race_year","team")\
        .agg(sum("points").alias("total_points"),
             count(when(col("position") == 1, True)).alias("wins"))

In [0]:
display(team_standings_df.filter("race_year=2020"))

race_year,team,total_points,wins


In [0]:
team_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc ("wins"))
team_standings_df = team_standings_df.withColumn("team_rank", rank().over(team_rank_spec))  

In [0]:
display(team_standings_df.filter("race_year=2020"))

race_year,team,total_points,wins,team_rank


In [0]:
##team_standings_df.write.mode("overwrite").parquet(F"{presentation_folder_path}/constructor_standings")
#team_standings_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standings")
#3 overwrite_partition(team_standings_df, "f1_presentation", "team_standings", "race_year")

In [0]:
merge_condition ="tgt.team =src.team AND tgt.race_year = src.race_year"
merge_delta_data (team_standings_df,"f1_presentation", "team_standings",presentation_folder_path,merge_condition,"race_year")